In [1]:
import sys
sys.path.append('../')

In [2]:
import game.agent as ag
from game.run import run
from game.simulator import load_simulator_yarn, GraphSimulator
from pathlib import Path
from tqdm.auto import trange
import pandas as pd
import torch

In [3]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

## Description

This game only has one personality trait, which is safety (no risk).

Therefore, a 1 implies a safe action (without risk), and a 0 implies a risky action.



In [4]:
n_runs = int(1e5)
# n_runs = int(100)
seed = None

## Simulator

In [5]:
simulator = load_simulator_yarn('../yarnScripts', graph_file_sfx='_graph_transformed.pickle')

## Agents

Values greater than 0 imply favoring safety

Values lower than 0 imply not favoring safety (favoring risk)

In [6]:
# label_predictor=ag.TorchLabelPredictor(
#             model_path=Path('./saved_bert/200_[20]_[30]_1_False_bert-base-multilingual-cased_0.001_adamw_8_max_val_mcc_False_False_100'),
#             use_cpu=False,
#         )

def get_linear_agent(a: torch.Tensor):
    """
    Gets an agent with a BehavioralDecisionMaker that has a linear function

    :param torch.Tensor a: matrix A that characterizes the behavior profile
    """
    return ag.LabelDecisorAgent(
        label_predictor=ag.GraphLabelLoader(
            simulator=simulator,
        ),
        decision_maker=ag.BehavioralDecisionMaker(
            weight_funcs=[
                lambda x: (a*x).sum()
            ],
            memory_steps=a.shape[0]-1,
            seed=None,
            deterministic=False,
        )
    )

In [7]:
def get_random_agent():
    return ag.RandomAgent(None)


# def get_r_agent():
#     return ag.LabelDecisorAgent(
#         label_predictor=ag.GraphLabelLoader(
#             simulator=simulator,
#         ),
#         decision_maker=ag.RDecisionMaker(
#             rand=0.4,
#             seed=None,
#         )
#     )


def get_risky_agent():
    return get_linear_agent(torch.as_tensor([
        [-2]
    ], dtype=torch.float))


def get_safe_agent():
    return get_linear_agent(torch.as_tensor([
        [2]
    ], dtype=torch.float))


def get_risky_ntm_agent():
    return get_linear_agent(torch.as_tensor([
        [-8],
        [8],
        [6],
    ], dtype=torch.float))

def get_safe_ntm_agent():
    return get_linear_agent(torch.as_tensor([
        [8],
        [-8],
        [-6],
    ], dtype=torch.float))


# def get_mc_agent():
#     return ag.LabelDecisorAgent(
#         label_predictor=ag.GraphLabelLoader(
#             simulator=simulator,
#         ),
#         decisor=ag.MarkovChainAgent(
#             rand=0.4,
#             seed=None,
#         )
#     )

In [8]:
# def get_random_agent():
#     return ag.RandomAgent(None)


# # def get_r_agent():
# #     return ag.LabelDecisorAgent(
# #         label_predictor=ag.GraphLabelLoader(
# #             simulator=simulator,
# #         ),
# #         decision_maker=ag.RDecisionMaker(
# #             rand=0.4,
# #             seed=None,
# #         )
# #     )


# def get_risky_agent():
#     return ag.LabelDecisorAgent(
#         label_predictor=ag.GraphLabelLoader(
#             simulator=simulator,
#         ),
#         decision_maker=ag.BehavioralDecisionMaker(
#             weight_funcs=[
#                 lambda x: x
#             ],
#             memory_steps=0,
#             seed=None,
#         )
#     )


# def get_risky_agent():
#     return ag.LabelDecisorAgent(
#         label_predictor=ag.GraphLabelLoader(
#             simulator=simulator,
#         ),
#         decision_maker=ag.BehavioralDecisionMaker(
#             weight_funcs=[
#                 lambda x: x
#             ],
#             memory_steps=0,
#             seed=None,
#         )
#     )


# def get_risky_ntm_agent():
#     return ag.LabelDecisorAgent(
#         label_predictor=ag.GraphLabelLoader(
#             simulator=simulator,
#         ),
#         decision_maker=ag.BehavioralDecisionMaker(
#             weight_funcs=[
#                 lambda x: 0.5*x,
#                 lambda x: -0.3*x,
#                 lambda x: -0.3*x,
#             ],
#             memory_steps=3,
#             seed=None,
#         )
#     )


# # def get_mc_agent():
# #     return ag.LabelDecisorAgent(
# #         label_predictor=ag.GraphLabelLoader(
# #             simulator=simulator,
# #         ),
# #         decisor=ag.MarkovChainAgent(
# #             rand=0.4,
# #             seed=None,
# #         )
# #     )

## Run

In [9]:
data = []
models = {
    'Random':0,
    'Risky':1,
    'Safe':2,
    'Risky NTM':3,
    'Safe NTM':4,
}

### Random Agent

In [ ]:
df_r = run(get_random_agent, simulator, n_runs)

In [ ]:
df = df_r

In [ ]:
df['endings']

In [ ]:
df['decisions'].groupby(['qid']).mean()
# diferencia entre estos valores y los otros nos dice si tiene predilección por label o no

In [ ]:
dec = df['decisions'].loc[df['decisions']['qid'] != -1,:]
dec

In [ ]:
dec = dec[['run','qid','label']].pivot(index='run', columns='qid', values='label').fillna(-1)
dec['m'] = models['Random']
data.append(dec)
dec

### Risky Agent

In [ ]:
df_risk = run(get_risky_agent, simulator, n_runs)

In [ ]:
df = df_risk

In [ ]:
df['endings']

In [ ]:
df['decisions'].groupby(['qid']).mean()
# diferencia entre estos valores y los otros nos dice si tiene predilección por label o no

In [ ]:
dec = df['decisions'].loc[df['decisions']['qid'] != -1,:]
dec

In [ ]:
dec = dec[['run','qid','label']].pivot(index='run', columns='qid', values='label').fillna(-1)
dec['m'] = models['Risky']
data.append(dec)
dec

### Risk Aversion (Safe) Agent

In [ ]:
df_no_risky = run(get_safe_agent, simulator, n_runs)

In [ ]:
df = df_no_risky

In [ ]:
df['endings']

In [ ]:
df['decisions'].groupby(['qid']).mean()
# diferencia entre estos valores y los otros nos dice si tiene predilección por label o no

In [ ]:
dec = df['decisions'].loc[df['decisions']['qid'] != -1,:]
dec

In [ ]:
dec = dec[['run','qid','label']].pivot(index='run', columns='qid', values='label').fillna(-1)
dec['m'] = models['No Risky']
data.append(dec)
dec

### Risky But Not Too Much Agent

In [ ]:
df_risk_ntm = run(get_risky_ntm_agent, simulator, n_runs)

In [ ]:
df = df_risk_ntm

In [ ]:
df['endings']

In [ ]:
df['decisions'].groupby(['qid']).mean()
# diferencia entre estos valores y los otros nos dice si tiene predilección por label o no

In [ ]:
dec = df['decisions'].loc[df['decisions']['qid'] != -1,:]
dec

In [ ]:
dec = dec[['run','qid','label']].pivot(index='run', columns='qid', values='label').fillna(-1)
dec['m'] = models['Risky NTM']
data.append(dec)
dec

### Safe But Not Too Much Agent

In [ ]:
df_safe_ntm = run(get_safe_ntm_agent, simulator, n_runs)

In [ ]:
df = df_safe_ntm

In [ ]:
df['endings']

In [ ]:
df['decisions'].groupby(['qid']).mean()
# diferencia entre estos valores y los otros nos dice si tiene predilección por label o no

In [ ]:
dec = df['decisions'].loc[df['decisions']['qid'] != -1,:]
dec

In [ ]:
dec = dec[['run','qid','label']].pivot(index='run', columns='qid', values='label').fillna(-1)
dec['m'] = models['No Risky NTM']
data.append(dec)
dec

## Data Analysis

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
# from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
# from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
# from sklearn.inspection import DecisionBoundaryDisplay
from tqdm.auto import tqdm

In [11]:
# df = pd.concat(data, ignore_index=True).fillna(-1)
# df.to_csv('results.csv', index=False)
df = pd.read_csv('results.csv')
df

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,14.0,15.0,17.0,m
0,-1.0,0.0,-1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,-1.0,-1.0,0.0,0
1,-1.0,0.0,-1.0,1.0,1.0,1.0,0.0,-1.0,0.0,1.0,1.0,0.0,-1.0,-1.0,1.0,0
2,-1.0,1.0,-1.0,1.0,0.0,-1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,-1.0,-1.0,0
3,-1.0,0.0,-1.0,1.0,0.0,-1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,-1.0,1.0,0
4,-1.0,0.0,-1.0,1.0,0.0,-1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,-1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,-1.0,0.0,-1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,4
499996,-1.0,0.0,-1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,4
499997,-1.0,0.0,-1.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,4
499998,-1.0,0.0,-1.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,4


In [12]:
n_classes = len(df['m'].unique())

In [13]:
x_train, x_test, y_train, y_test = train_test_split(df.loc[:,df.columns!='m'], df['m'], test_size=0.3)

In [14]:
y_train.value_counts(normalize=True)

4    0.200251
3    0.200234
1    0.200060
0    0.199926
2    0.199529
Name: m, dtype: float64

In [15]:
y_test.value_counts(normalize=True)

2    0.201100
0    0.200173
1    0.199860
3    0.199453
4    0.199413
Name: m, dtype: float64

In [16]:
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
x_test = x_test.to_numpy()
y_test = y_test.to_numpy()

In [19]:
dt = DecisionTreeClassifier()
dt.fit(x_train, y_train)

lr = LogisticRegression(max_iter=500)
lr.fit(x_train, y_train)

LogisticRegression(max_iter=500)

In [20]:
dt.score(x_test,y_test)

0.7466133333333334

In [21]:
y_pred = dt.predict(x_test)
print(metrics.classification_report(y_true=y_test,y_pred=y_pred, digits=3))
print(metrics.confusion_matrix(y_true=y_test,y_pred=y_pred, normalize='true').round(3))
print(metrics.matthews_corrcoef(y_true=y_test,y_pred=y_pred).round(3))

              precision    recall  f1-score   support

           0      0.615     0.511     0.558     29641
           1      0.817     0.806     0.812     30115
           2      0.664     0.843     0.743     30278
           3      0.802     0.894     0.845     30030
           4      0.852     0.674     0.753     29936

    accuracy                          0.747    150000
   macro avg      0.750     0.746     0.742    150000
weighted avg      0.750     0.747     0.743    150000

[[0.511 0.127 0.168 0.141 0.052]
 [0.091 0.806 0.001 0.068 0.034]
 [0.112 0.    0.843 0.013 0.031]
 [0.08  0.021 0.005 0.894 0.   ]
 [0.032 0.034 0.259 0.    0.674]]
0.686


In [22]:
lr.score(x_test,y_test)

0.6773533333333334

In [23]:
# https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html
# https://scikit-learn.org/stable/auto_examples/neighbors/plot_nca_dim_reduction.html

In [24]:
# x,y = x_test,y_test

# pca = PCA(n_components=2)
# Xreduced = pca.fit_transform(x)
# c = DecisionTreeClassifier()
# c.fit(Xreduced,y)

# def make_meshgrid(x, y, h=.02):
#     x_min, x_max = x.min() - 1, x.max() + 1
#     y_min, y_max = y.min() - 1, y.max() + 1
#     xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
#     return xx, yy

# def plot_contours(ax, clf, xx, yy, **params):
#     Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
#     Z = Z.reshape(xx.shape)
#     out = ax.contourf(xx, yy, Z, **params)
#     return out

# fig, ax = plt.subplots(figsize=(15,15))
# # title for the plots
# title = ('Decision surface of linear SVC ')
# # Set-up grid for plotting.
# X0, X1 = Xreduced[:, 0], Xreduced[:, 1]
# xx, yy = make_meshgrid(X0, X1)

# plot_contours(ax, c, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
# ax.scatter(X0, X1, c=y, cmap=plt.cm.coolwarm, s=20, edgecolors='k')
# ax.set_ylabel('PC2')
# ax.set_xlabel('PC1')
# ax.set_xticks(())
# ax.set_yticks(())
# ax.set_title('Decison surface using the PCA transformed/projected features')
# ax.legend()
# plt.show()

In [21]:
c_names = [
    'DT',
    'LR',
    'Ada',
    'RF',
    'KNN',
    'SVC',
]

classifiers= [
    lambda : DecisionTreeClassifier(),
    lambda : LogisticRegression(max_iter=500),
    lambda : AdaBoostClassifier(),
    lambda : RandomForestClassifier(),
    lambda : KNeighborsClassifier(n_neighbors=n_classes),
    lambda : SVC(kernel='linear'),
    # SVC(),    
]

In [19]:
m = []
for cf in tqdm(classifiers):
    # print(f"{color.BLUE}{color.BOLD}Classifier {c}{color.END}")
    
    # train model
    c = cf()
    c.fit(x_train, y_train)
    y_pred = c.predict(x_test)

    # f1 scores
    rep = metrics.classification_report(y_true=y_test,y_pred=y_pred, output_dict=True)
    f1 = [rep[str(k)]['f1-score'] for k in models.values()] + [metrics.f1_score(y_true=y_test,y_pred=y_pred, average='weighted')]

    # accuracy
    cm = metrics.confusion_matrix(y_true=y_test,y_pred=y_pred, normalize='true')
    acc = cm.diagonal().tolist() + [rep['accuracy']]

    m.append(pd.DataFrame(data=[f1, acc], columns=list(models.keys()) + ['global'], index=['f1', 'accuracy']).round(3))
    
    # # get evaluation info
    # y_pred = c.predict(x_test)
    # print(metrics.classification_report(y_true=y_test,y_pred=y_pred, digits=3))
    # print(metrics.confusion_matrix(y_true=y_test,y_pred=y_pred, normalize='true').round(3))
    # print(metrics.f1_score(y_true=y_test,y_pred=y_pred, average='weighted').round(3))
    # print(metrics.matthews_corrcoef(y_true=y_test,y_pred=y_pred).round(3))

  0%|          | 0/5 [00:00<?, ?it/s]

Classifier <function <lambda> at 0x00000243CE5EC670>


 20%|██        | 1/5 [00:02<00:10,  2.58s/it]

Classifier DecisionTreeClassifier()


 40%|████      | 2/5 [01:09<02:01, 40.59s/it]

...

 60%|██████    | 3/5 [01:45<01:16, 38.26s/it]

Classifier AdaBoostClassifier()


 80%|████████  | 4/5 [02:30<00:40, 40.87s/it]

Classifier RandomForestClassifier()


100%|██████████| 5/5 [05:44<00:00, 68.93s/it]


In [24]:
df_m = pd.concat(m, keys=c_names)
# df_m.to_excel('results_metrics.xlsx')
df_m

Random  Risky  No Risk  Risky NTM  No Risky NTM  global
DT  f1         0.557  0.809    0.744      0.843         0.758   0.742
    accuracy   0.505  0.811    0.845      0.889         0.683   0.747
LR  f1         0.438  0.785    0.716      0.760         0.654   0.670
    accuracy   0.376  0.759    0.781      0.808         0.667   0.678
Ada f1         0.445  0.785    0.733      0.748         0.668   0.676
    accuracy   0.388  0.757    0.823      0.799         0.650   0.684
RF  f1         0.557  0.809    0.744      0.843         0.758   0.742
    accuracy   0.505  0.810    0.845      0.889         0.683   0.747
KNN f1         0.518  0.772    0.720      0.812         0.747   0.714
    accuracy   0.496  0.758    0.808      0.826         0.688   0.715